In [50]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch


In [51]:
class HAM10000Dataset(Dataset):
    def __init__(self, csv_file, img_dir, file_prefix = "", transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.file_prefix = file_prefix
        self.transform = transform
        self.classes = sorted(self.data.iloc[:, 2].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.file_prefix + self.data.iloc[idx, 1] + '.jpg')
        image = Image.open(img_name)
        label = self.data.iloc[idx, 2]
        label_idx = self.class_to_idx[label]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label_idx, dtype=torch.long)

In [52]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = HAM10000Dataset(csv_file='data/archive/HAM10000_metadata.csv', img_dir='data/hair_removed/', file_prefix="nohair_", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [53]:
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights

def create_model(num_classes=7):
    # todo: what weights to use?
    model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1)

    # todo: modify the classifier?
    model.classifier[2] = torch.nn.Linear(768, num_classes)

    return model

num_classes = len(dataset.classes)
model = create_model(num_classes=num_classes)

if torch.cuda.is_available():
    model = model.cuda()

In [54]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)

In [55]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()

        if isinstance(labels, tuple):
            labels = labels[0]

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

In [ ]:
model.eval()
correct = 0
total = 0

idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

with torch.no_grad():
    for images, labels in dataloader:
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Convert numeric predictions back to string labels if needed
        predicted_labels = [idx_to_class[idx.item()] for idx in predicted]
        true_labels = [idx_to_class[idx.item()] for idx in labels]

        print(f"Predicted: {predicted_labels}")
        print(f"Actual: {true_labels}")

print(f'Accuracy: {100 * correct / total}%')